# Put in your name below for merging the dataset later

In [ ]:
name = "V"

In [ ]:
filename = f"{name}_screen_"

In [3]:
import xml
import cv2
import subprocess
import matplotlib.pyplot as plt
import sys

from pathlib import Path


from ppadb.client import Client as AdbClient
import xml.dom.minidom
import re

In [4]:
xml_annotations_path = Path("../xml_annotations/")
annotations_path = Path("../annotations/")
screenshots_path = Path("../screenshots/")

In [5]:
plt.style.use("seaborn-dark")
plt.rcParams["figure.figsize"] = 4.5, 8

# Get XML and screenshot from android

In [6]:
last_file_index = 0

if annotations_path.exists():
    existing_files = sorted(screenshots_path.glob("*.png"))

    if existing_files:
        last_file = existing_files[-1]
        last_file_index = int(re.findall(r"(\d+)", last_file.stem)[-1])

In [7]:
client = AdbClient(host="127.0.0.1", port=5037)
devices = client.devices()
for device in devices:
    print(device.serial)

192.168.0.24:5555


In [8]:
device = client.device("192.168.0.24:5555")

> Important

## Run the cell below after changing every screen to get screenshot and xml

In [12]:
# store_screenshot
device.shell("screencap -p /sdcard/screen.png")
device.pull("/sdcard/screen.png", (screenshots_path/f"{filename}_{last_file_index:04}.png"))

# store full layout (Raw)
device.shell("uiautomator dump")
device.pull("/sdcard/window_dump.xml", (xml_annotations_path/f"full_{filename}_{last_file_index:04}.xml"))

# store compressed (Relevant)
device.shell("uiautomator dump --compressed")
device.pull("/sdcard/window_dump.xml", (xml_annotations_path/f"{filename}_{last_file_index:04}.xml"))


last_file_index += 1